⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.

In [1]:
# Project path setup
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project root added:", PROJECT_ROOT)


✅ Project root added: C:\Users\shubh\crypto-market-opportunity-engine


In [2]:
import pandas as pd
import numpy as np

import src.config as config


In [3]:
DATA_SIGNALS = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_signals.parquet"

df = pd.read_parquet(DATA_SIGNALS)

print("Shape:", df.shape)
df[["close", "prob_up", "signal"]].head()


Shape: (105092, 28)


,close,prob_up,signal
open_time,,,
2025-01-01 02:20:00,93716.81,0.503679,HOLD
2025-01-01 02:25:00,93742.52,0.504466,HOLD
2025-01-01 02:30:00,93773.61,0.503574,HOLD
2025-01-01 02:35:00,93879.99,0.499005,HOLD
2025-01-01 02:40:00,93921.86,0.492561,HOLD


In [4]:
# risk configuration
INITIAL_CAPITAL = 100_000
RISK_PER_TRADE = 0.01      # 1%
STOP_LOSS_PCT = 0.01       # 1%
TAKE_PROFIT_PCT = 0.02     # 2%
FEE_RATE = 0.0004          # Binance-like


In [5]:
# trade state columns
df = df.copy()

df["position"] = 0
df["position_size"] = 0.0
df["entry_price"] = np.nan
df["stop_loss"] = np.nan
df["take_profit"] = np.nan
df["pnl"] = 0.0
df["capital"] = INITIAL_CAPITAL


In [6]:
# risk aware trading logic
capital = INITIAL_CAPITAL
position = 0
entry_price = 0
position_size = 0

for i in range(len(df)):
    price = df.iloc[i]["close"]
    signal = df.iloc[i]["signal"]

    # =====================
    # ENTER TRADE
    # =====================
    if signal == "BUY" and position == 0:
        risk_amount = capital * RISK_PER_TRADE
        stop_loss = price * (1 - STOP_LOSS_PCT)
        take_profit = price * (1 + TAKE_PROFIT_PCT)

        position_size = risk_amount / (price - stop_loss)
        entry_price = price
        position = 1

        df.iloc[i, df.columns.get_loc("position")] = 1
        df.iloc[i, df.columns.get_loc("position_size")] = position_size
        df.iloc[i, df.columns.get_loc("entry_price")] = price
        df.iloc[i, df.columns.get_loc("stop_loss")] = stop_loss
        df.iloc[i, df.columns.get_loc("take_profit")] = take_profit

    # =====================
    # EXIT TRADE
    # =====================
    elif position == 1:
        if price <= stop_loss or price >= take_profit or signal == "SELL":
            pnl = (price - entry_price) * position_size
            pnl -= abs(pnl) * 2 * FEE_RATE

            capital += pnl
            position = 0
            position_size = 0

            df.iloc[i, df.columns.get_loc("pnl")] = pnl

    df.iloc[i, df.columns.get_loc("capital")] = capital


In [7]:
# equity curve
df["equity_curve"] = df["capital"] / INITIAL_CAPITAL
df[["capital", "equity_curve"]].tail()


,capital,equity_curve
open_time,,
2025-12-31 23:35:00,125410.780822,1.254108
2025-12-31 23:40:00,125410.780822,1.254108
2025-12-31 23:45:00,125410.780822,1.254108
2025-12-31 23:50:00,125410.780822,1.254108
2025-12-31 23:55:00,125410.780822,1.254108


In [8]:
# risk performance metrics
trades = df[df["pnl"] != 0]

total_return = df["equity_curve"].iloc[-1] - 1
win_rate = (trades["pnl"] > 0).mean()
max_drawdown = (df["equity_curve"] / df["equity_curve"].cummax() - 1).min()

sharpe = (
    df["capital"].pct_change().mean()
    / df["capital"].pct_change().std()
) * np.sqrt(365 * 24 * 12)

print(f"📈 Total Return: {total_return:.2%}")
print(f"🎯 Win Rate: {win_rate:.2%}")
print(f"📉 Max Drawdown: {max_drawdown:.2%}")
print(f"⚖️ Sharpe Ratio: {sharpe:.2f}")
print(f"🔁 Trades: {len(trades)}")


📈 Total Return: 25.41%
🎯 Win Rate: 52.94%
📉 Max Drawdown: -3.26%
⚖️ Sharpe Ratio: 2.09
🔁 Trades: 34


In [9]:
# sanity checks
trades[["entry_price", "stop_loss", "take_profit", "pnl"]].head(10)


,entry_price,stop_loss,take_profit,pnl
open_time,,,,
2025-01-08 11:35:00,NaN,NaN,NaN,-1044.896420
2025-01-09 08:05:00,NaN,NaN,NaN,-1103.197109
2025-01-10 17:45:00,NaN,NaN,NaN,2062.166983
2025-01-16 16:05:00,NaN,NaN,NaN,2164.699866
2025-01-20 05:05:00,NaN,NaN,NaN,2198.929032
2025-01-20 17:50:00,NaN,NaN,NaN,2158.580768
2025-01-26 23:05:00,NaN,NaN,NaN,-1130.429196
2025-02-03 07:45:00,NaN,NaN,NaN,2535.936984
2025-02-10 00:05:00,NaN,NaN,NaN,2290.053595


In [10]:
DATA_RISK = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_risk_managed.parquet"

df.to_parquet(DATA_RISK)
print("✅ Risk-managed data saved")


✅ Risk-managed data saved
